# Advanced examples of DMFF 1.0.0
In our new tutorial notebook https://nb.bohrium.dp.tech/detail/6366839940 You must already have a basic understanding of DMFF version 1.0.0. As an advanced tutorial, we have also prepared this example-notebook for you as a supplement to the Tutorial, which includes an introduction to new modules in DMFF such as Qeq, ML Force, and the OpenMM plugin.

## Environment Setup

Retrieve DMFF from GitHub and switch to the desired branch, then proceed with the installation.

In [1]:
! rm -rf DMFF
! rm -rf /opt/mamba/lib/python3.10/site-packages/dmff*
! git clone https://github.com/deepmodeling/DMFF.git
! git config --global --add safe.directory `pwd`/DMFF
! cd DMFF && git checkout wangxy/v1.0.0-devel && pip install .

Cloning into 'DMFF'...
remote: Enumerating objects: 4439, done.
remote: Counting objects: 100% (4439/4439), done.
remote: Compressing objects: 100% (1432/1432), done.
remote: Total 4439 (delta 2961), reused 4361 (delta 2922), pack-reused 0
Receiving objects: 100% (4439/4439), 22.10 MiB | 5.00 MiB/s, done.
Resolving deltas: 100% (2961/2961), done.
Updating files: 100% (273/273), done.
Updating files: 100% (318/318), done.
Branch 'wangxy/v1.0.0-devel' set up to track remote branch 'wangxy/v1.0.0-devel' from 'origin'.
Switched to a new branch 'wangxy/v1.0.0-devel'
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Processing /data/DMFF
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.3 MB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 3.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.3/170.3 kB 3.9 MB/s eta 0:00:00a 0:00:01
  Created wheel for dmff: filename=dmff-0

Install the required libraries; this step is time-consuming, so please be patient.

In [2]:
! mamba install openmm=7.7.0 rdkit -c conda-forge -y
! pip install parmed mdtraj pymbar networkx


                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (0.27.0) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['openmm=7.7.0', 'rdkit']

[+] 0.0s
[+] 0.1s
conda-forge/linux-64 ━━━━━━━━━━━━━╸━━━━━━━━━   0.0 B /

## 1. ADMPQeqForce

ADMPQeqForce provides a support to coulombic energy calculation for constant potential model and constant charge model. Net charges on all atoms were equilibrated at setted constraint first, then charge related energys were carried out next.

You can directly run the test:

In [5]:
run DMFF/tests/test_admp/test_qeq.py

And we will provide a more detailed explanation as follows

### Import the necessary libraries

In [6]:
import openmm.app as app
import openmm.unit as unit
from dmff.api import Hamiltonian
from dmff.api import DMFFTopology
from dmff.api.xmlio import XMLIO
from dmff import NeighborList
import jax
from jax import value_and_grad
import jax.numpy as jnp
import numpy as np
import time
import pickle
import sys

### Load your force field

In [7]:
xml = XMLIO()
xml.loadXML("DMFF/tests/data/qeq2.xml")

# get residues
res = xml.parseResidues()

For information about the force field file, please refer to the user guide, which contains detailed explanations.
### Initialize the charge and type of each atom and aux

In [8]:
charges, types = [], []
for i in range(len(res)):
    charges += [a["charge"] for a in res[i]["particles"]]
    types += [a["type"] for a in res[i]["particles"]]
charges = np.zeros((len(charges),))

# initialize aux
aux = {
    "q": jnp.array(charges),
     "lagmt": jnp.array([1.0, 1.0])
    #"lagmt": jnp.array([1.0])
}

### Load the topological information and supplement it

In [9]:
# Load topology
pdb = app.PDBFile("DMFF/tests/data/qeq2.pdb")
dmfftop = DMFFTopology(from_top=pdb.topology)
pos = pdb.getPositions(asNumpy=True).value_in_unit(unit.nanometer)
pos = jnp.array(pos)
box = dmfftop.getPeriodicBoxVectors()

# Assign atom charges and types in te topology
atoms = [a for a in dmfftop.atoms()]
for na, a in enumerate(atoms):
    a.meta["charge"] = charges[na]
    a.meta["type"] = types[na]

### Preparation for potential function

In [10]:
# create Hamiltonian
hamilt = Hamiltonian("DMFF/tests/data/qeq2.xml")

# create neighborlist & pairs
nblist = NeighborList(box, 0.6, dmfftop.buildCovMat())
pairs = nblist.allocate(pos)         

# initialize const_list
const_list, map_atomtype = [], []
for i in dmfftop.residues():
    temp = []
    for j in i.atoms():
        temp.append(int(j.id)-1)
    const_list.append(np.array(temp))

# create map_atomtype
for i in dmfftop.atoms():
    map_atomtype.append(int(i.meta["type"])-1)    #temp set

# assign const_val
n_template = len(const_list)
const_val = jnp.zeros(n_template)

### Create potential function and Calculate the energy

In [11]:
pot = hamilt.createPotential(dmfftop, nonbondedCutoff=0.6*unit.nanometer, nonbondedMethod=app.PME,
                            ethresh=1e-3, neutral=True, slab=False, constQ=True,
                            const_list=const_list, const_vals=const_val,
                            has_aux=True)

#return energy
efunc = pot.getPotentialFunc()
energy, aux = efunc(pos, box, pairs, hamilt.paramset.parameters, aux)
print("energy: %f kj/mol" %energy)
print(aux)

energy: 4817.286675 kj/mol
{'q': DeviceArray([-2.99605719e-04, -3.40972179e-04, -4.91927203e-04,
             -7.57415141e-04, -9.72305199e-04, -9.04476306e-04,
             -6.19403852e-04, -3.18511669e-04, -4.14033308e-04,
             -5.08883423e-04, -5.64320831e-04, -5.92336096e-04,
             -4.59863367e-04, -5.31103425e-04, -3.93029108e-04,
             -4.28646761e-04, -2.86954295e-04, -3.30793706e-04,
             -3.17696799e-04, -3.50221376e-04, -2.77981466e-04,
             -3.21483470e-04, -2.96629949e-04, -3.72879359e-04,
             -3.51565779e-04, -4.66407859e-04, -5.10171801e-04,
              3.04087838e-04, -3.70321220e-03,  3.20495493e-04,
             -9.18942861e-04, -3.34922291e-04, -4.97683690e-04,
             -3.47713379e-04, -3.55104058e-04, -2.89328710e-04,
             -3.24275491e-04, -2.80456324e-04, -3.02740626e-04,
             -2.75075995e-04, -2.91228744e-04, -2.70938814e-04,
             -4.46974131e-04, -2.97128120e-04, -8.78905378e-04,
       

## 2. Machine Learning Force

## 2.1 SGNN
Navigate to the working directory

In [12]:
import os
os.chdir(os.path.join("DMFF","examples", "sgnn"))

SGNN assume the remaining bonding energy can be written as a sum over different local fragments of the molecule. These fragments are defined as “subgraphs” (labeled as g):

$$
E_{sGNN}=\sum {E_{g}}
$$

Each subgraph defines the local environment of a central bond, and $E_g$ represents the intramolcular energy attributed to that bond. This leads to a rigorously localized representation of the molecule, warranting the extendibility of the resulting model.

### Create a SGNN potential function

For information about the force field file, please refer to the user guide, which contains detailed explanations. Now you need to do the following to create a SGNN potential:

In [13]:
H = Hamiltonian('peg.xml')
app.Topology.loadBondDefinitions("residues.xml")
pdb = app.PDBFile("peg4.pdb")
rc = 0.6
# generator stores all force field parameters
pots = H.createPotential(pdb.topology, nonbondedCutoff=rc*unit.nanometer, ethresh=5e-4)

### Preparation for energy calculation

In [14]:
# construct inputs
positions = jnp.array(pdb.positions._value)
a, b, c = pdb.topology.getPeriodicBoxVectors()
box = jnp.array([a._value, b._value, c._value])
# neighbor list
nbl = NeighborList(box, rc, pots.meta['cov_map']) 
nbl.allocate(positions)

DeviceArray([[ 0,  1,  1],
             [ 0,  2,  1],
             [ 0,  3,  1],
             [ 0,  4,  2],
             [ 0,  5,  3],
             [ 0,  6,  3],
             [ 0,  7,  3],
             [ 0,  8,  3],
             [ 0,  9,  4],
             [ 0, 10,  4],
             [ 0, 11,  2],
             [ 0, 12,  1],
             [ 0, 13,  2],
             [ 0, 14,  2],
             [ 0, 19,  4],
             [ 0, 20,  5],
             [ 0, 21,  5],
             [ 1,  2,  2],
             [ 1,  3,  2],
             [ 1,  4,  3],
             [ 1,  5,  4],
             [ 1,  6,  4],
             [ 1,  7,  4],
             [ 1,  8,  4],
             [ 1,  9,  5],
             [ 1, 10,  5],
             [ 1, 11,  3],
             [ 1, 12,  2],
             [ 1, 13,  3],
             [ 1, 14,  3],
             [ 1, 19,  5],
             [ 1, 20,  6],
             [ 1, 21,  6],
             [ 2,  3,  2],
             [ 2,  4,  3],
             [ 2,  5,  4],
             [ 2,  6,  4],
 

And you can get parameters by:

In [15]:
paramset = H.getParameters()

### Load data and fix it

In [16]:
with open('test_backend/set_test_lowT.pickle', 'rb') as ifile:
    data = pickle.load(ifile)

# input in nm
pos = jnp.array(data['positions'][0:20]) / 10
box = jnp.eye(3) * 5

### Calculate the energy

In [17]:
efunc = jax.jit(pots.getPotentialFunc())
efunc_vmap = jax.vmap(jax.jit(pots.getPotentialFunc()), in_axes=(0, None, None, None), out_axes=0)
print(efunc(pos[0], box, nbl.pairs, paramset))
print(efunc_vmap(pos, box, nbl.pairs, paramset))

-21.588284621154514
[-21.58828462 -39.79334159  10.03889335 -48.22451239 -32.90970162
 -49.68568287 -47.58035178 -51.73860617 -37.39235277 -35.01933271
 -46.06621902 -31.69327601  -6.86739655  -5.13698524 -27.4031207
 -44.65301991 -52.00357797   3.1734038  -72.79081259 -28.27007722]


## 2.1 EANN
Navigate to the working directory

In [18]:
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
os.chdir(parent_directory)
os.chdir(os.path.join("eann"))

EANN framework born out from the EAM idea. This physically inspired embedded atom neuralnetworks (EANN) representation is not only conceptually andnumerically simple but also very efficient and accurate, as discussed below. EANN assume that the impurity experiences a locally uniform electron density, the embedding energy can be approximated as a function of the scalar local electron density at the impurity site plus an electrostatic interaction. Considering all atoms in the system as impurities embedded in the electron gas created by other atoms, in the EAM framework, the total energy of an $N$ atom system is just the sum over all individual impurity energies.

$$
E=\sum_{i=1}^{N} E_{i}=\sum_{i=1}^{N}\left[F_{i}\left(\rho_{i}\right)+\frac{1}{2} \sum_{j \neq i} \phi_{i j}\left(r_{i j}\right)\right]
$$

where $F_i$ is the embedding function, $ρ_i$ is the embedded electron density at the position of atom $i$ given by the superposition of the densities of surrounding atoms, and $\phi_{ij}$ is the short-range repulsive potential between atoms $i$ and $j$ depending on their distance $r_{ij}$. As the exact forms of these functions are generally unknown, they are often taken from electron gas computations or fit to experimental properties with semiempirical expres-sions. Given these intrinsic approximations, EAM or even its modified version has a limited accuracy and is mainly suitablefor metallic systems.

To go beyond the EAM, we need to improve both expressions of the embedded density and the function $F$. To this end, EANN start from the commonly used Gaussian-type orbitals (GTOs) centered at each atom,

$$
\phi_{l_{x} y_{l} y_{z}}^{\alpha, r_{s}}=x^{l_{x}} y^{l_{y}} z^{l_{z}} \exp \left(-\alpha\left|r-r_{s}\right|^{2}\right)
$$

where each atom is taken as the origin, $r=(x,y,z)$ constitutes the coordinate vector of an electron, $r$ is the norm of the vector,$α$ and $r_s$ are parameters that determine radial distributions of atomic orbitals, ${l_x+l_y+l_z=L}$ specifies the orbital angular momentum ($L$), e.g., $L$ = 0, 1, and 2, correspond to the s, p, and d orbitals, respectively. In this representation, the embedded density of atom $i$ can be taken as the square of the linear combination of atomic orbitals from neighboring atoms, in a similar spirit as that in Hartree−Fock (HF) and densityfunctional theory (DFT). This would generate a scalar $ρ^i$ value for the embedding atom $i$, as used in the EAM, which has been proven to offer insufficient representability for the total energyand can be improved by including the gradients of density.

As for code, just follow the step in SGNN:

In [19]:
H = Hamiltonian('peg.xml')
app.Topology.loadBondDefinitions("residues.xml")
pdb = app.PDBFile("peg4.pdb")
rc = 0.4
# generator stores all force field parameters
pots = H.createPotential(pdb.topology, nonbondedCutoff=rc*unit.nanometer, ethresh=5e-4)

# construct inputs
positions = jnp.array(pdb.positions._value)
a, b, c = pdb.topology.getPeriodicBoxVectors()
box = jnp.array([a._value, b._value, c._value])
# neighbor list
nbl = NeighborList(box, rc, pots.meta['cov_map']) 
nbl.allocate(positions)


paramset = H.getParameters()
# params = paramset.parameters
paramset.parameters

efunc = jax.jit(pots.getPotentialFunc())
print(efunc(positions, box, nbl.pairs, paramset))

-0.09797672247941636


## 3. OpenMM Plugin for DMFF

OpenMM DMFF plugin was developed for [OpenMM](http://openmm.org) to incorporate the trained JAX model from [DMFF](https://github.com/deepmodeling/DMFF) as an independent Force class for molecular dynamics simulations.
To utilize this plugin, you need to save your DMFF model using the `DMFF/backend/save_dmff2tf.py` script.
The `save_dmff2tf.py` script converts the DMFF model to a TensorFlow module using the experimental feature of JAX called [`jax2tf`](https://github.com/google/jax/blob/main/jax/experimental/jax2tf/README.md).
The integration of the saved TensorFlow module with the DMFF plugin is accomplished using [cppflow](https://github.com/serizba/cppflow) and the OpenMM C++ interface. 

**Here you might need to shut down the original node and start up a new one**

### Save the DMFF model with `save_dmff2tf.py` script

#### Install TensorFlow and JAX

In [2]:
import os
os.chdir('/data')

In [3]:
!pip install "jax[cpu]==0.4.14"
!pip install tensorflow

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 5.6 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.7/206.7 kB 4.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.7/73.7 MB 15.3 MB/s eta 0:00:0000:0100:01
  Created wheel for jax: filename=jax-0.4.14-py3-none-any.whl size=1535363 sha256=87520dcb5f85c5073802ec72c4583f7acda7b09c0432e41f25a1d6ab1b34a94e
  Stored in directory: /root/.cache/pip/wheels/7d/7a/8d/c75c9181b2ea75b6953c3a91fdacfec166038cb6d815807149
Successfully built jax
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.3.15+cuda11.cudnn82
    Uninstalling jaxlib-0.3.15+cuda11.cudnn82:
      Successfully uninstalled jaxlib-0.3.15+cuda11.cudnn82
  Attempting uninstall: jax
    Found existing installation

#### Save DMFF with dmff2tf script

In [ ]:
!python DMFF/backend/save_dmff2tf.py --input_pdb DMFF/examples/water_fullpol/water_dimer.pdb --xml_files DMFF/examples/water_fullpol/forcefield.xml --output /tmp/dmff_admp_water_dimer --has_aux True
!ls /tmp/dmff_admp_water_dimer

!python DMFF/backend/save_dmff2tf.py --input_pdb DMFF/examples/classical/lig.pdb --xml_files DMFF/examples/classical/lig-prm.xml --output /tmp/dmff_classical_lig

### Install OpenMM DMFF Plugin

In [5]:
!mamba create -n dmff_omm -c conda-forge -y python=3.9 openmm libtensorflow_cc=2.9.1 swig=4.0.1 setuptools=59.5.0
!mamba activate dmff_omm


                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (0.27.0) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['python=3.9', 'openmm', 'libtensorflow_cc=2.9.1', 'swig=4.0.1', 'setuptools=59.5.0']

[+] 0.0s
[+]

In [6]:
!wget https://github.com/tensorflow/tensorflow/archive/refs/tags/v2.9.1.tar.gz
!tar -xf v2.9.1.tar.gz --no-same-owner
!cp -r tensorflow-2.9.1/tensorflow/c /opt/mamba/envs/dmff_omm/include/tensorflow

--2023-11-09 06:26:46--  https://github.com/tensorflow/tensorflow/archive/refs/tags/v2.9.1.tar.gz
Resolving ga.dp.tech (ga.dp.tech)... 10.255.254.7, 10.255.254.37, 10.255.254.18
Connecting to ga.dp.tech (ga.dp.tech)|10.255.254.7|:8118... connected.
Proxy request sent, awaiting response... 302 Found
Location: https://codeload.github.com/tensorflow/tensorflow/tar.gz/refs/tags/v2.9.1 [following]
--2023-11-09 06:26:47--  https://codeload.github.com/tensorflow/tensorflow/tar.gz/refs/tags/v2.9.1
Connecting to ga.dp.tech (ga.dp.tech)|10.255.254.7|:8118... connected.
Proxy request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘v2.9.1.tar.gz’

v2.9.1.tar.gz           [                 <=>]  63.57M  5.04MB/s    in 14s     

2023-11-09 06:27:02 (4.66 MB/s) - ‘v2.9.1.tar.gz’ saved [66654318]



In [7]:
!git clone https://github.com/serizba/cppflow.git
!cd /data/cppflow && git apply /data/DMFF/backend/openmm_dmff_plugin/tests/cppflow_empty_constructor.patch
!mkdir /opt/mamba/envs/dmff_omm/include/cppflow
!cd /data/cppflow && cp -r include/cppflow /opt/mamba/envs/dmff_omm/include/

Cloning into 'cppflow'...
remote: Enumerating objects: 1011, done.
remote: Counting objects: 100% (299/299), done.
remote: Compressing objects: 100% (138/138), done.
remote: Total 1011 (delta 182), reused 237 (delta 157), pack-reused 712
Receiving objects: 100% (1011/1011), 8.70 MiB | 5.32 MiB/s, done.
Resolving deltas: 100% (483/483), done.


In [8]:
!cd /data/DMFF/backend/openmm_dmff_plugin/ && mkdir build && cd build && cmake .. -DOPENMM_DIR=/opt/mamba/envs/dmff_omm -DCPPFLOW_DIR=/opt/mamba/envs/dmff_omm -DTENSORFLOW_DIR=/opt/mamba/envs/dmff_omm -DSWIG_EXECUTABLE=/opt/mamba/envs/dmff_omm/bin/swig -DPYTHON_EXECUTABLE=/opt/mamba/envs/dmff_omm/bin/python && make && make install && make PythonInstall

-- The C compiler identification is GNU 9.4.0
-- The CXX compiler identification is GNU 9.4.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Failed
-- Looking for pthread_create in pthreads
-- Looking for pthread_create in pthreads - not found
-- Looking for pthread_create in pthread
-- Looking for pthread_create in pthread - found
-- Found Threads: TRUE  
-- CUDA found, building CUDA implementatio

In [9]:
!cd /data/DMFF/backend && /opt/mamba/envs/dmff_omm/bin/python -m OpenMMDMFFPlugin.tests.test_dmff_plugin_nve -n 100 --pdb ../examples/water_fullpol/water_dimer.pdb --model ./openmm_dmff_plugin/python/OpenMMDMFFPlugin/data/admp_water_dimer_aux --has_aux True

2023-11-09 06:35:31.878151: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: ./openmm_dmff_plugin/python/OpenMMDMFFPlugin/data/admp_water_dimer_aux
2023-11-09 06:35:32.138075: I tensorflow/cc/saved_model/reader.cc:81] Reading meta graph with tags { serve }
2023-11-09 06:35:32.138139: I tensorflow/cc/saved_model/reader.cc:122] Reading SavedModel debug info (if present) from: ./openmm_dmff_plugin/python/OpenMMDMFFPlugin/data/admp_water_dimer_aux
2023-11-09 06:35:32.139209: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-09 06:35:32.854369: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2023-11-09 06:35:32.946199: I ten